In [1]:
import datetime
import logging
import os
import numpy as np
from ocean_navigation_simulator.reinforcement_learning.runners.GenerationRunner import (
    GenerationRunner,
)
import matplotlib.pyplot as plt
## Only when developing with VSCode in my repo 
#os.chdir('/home/nicolas/codeRepo/OceanPlatformControl') # here in Azure
# os.chdir('/home/nicolas/documents/Master_Thesis_repo/OceanPlatformControl') # here in WSL
# print(os.getcwd())
##
import pandas as pd
from ocean_navigation_simulator.utils.units import Distance
# These lines in VSCode to avoid reloading the kernel when changes are made to the external kernel
%load_ext autoreload
%autoreload 2

### Define Filenames

In [2]:
path_to_files = 'generated_media/metrics_csv_results/'
#file_containing_pb = 'problemsGOM.csv'
pb_name_hj_naive = 'multi_ag_4_HC_HC_hj_naive_549.csv'
pb_name_reactive = 'multi_ag_4_HC_HC_reactive_control_548.csv'
pb_name_flocking = 'multi_ag_4_HC_HC_flocking_548.csv'

### Plot problem distribution

In [3]:
# TODO

### Extract metrics from csv files

In [3]:
metrics_hj_naive_unfiltered = pd.read_csv(f"{path_to_files}/{pb_name_hj_naive}")
metrics_hj_reactive_unfiltered = pd.read_csv(f"{path_to_files}/{pb_name_reactive}")
metrics_hj_flocking_unfiltered = pd.read_csv(f"{path_to_files}/{pb_name_flocking}")

+ Make sure we have the (same) indices and total number of missions for the panda dataframes

In [4]:
# Find the common mission_nr values
common_mission_nr = set(metrics_hj_naive_unfiltered['mission_nr']).intersection(set(metrics_hj_reactive_unfiltered['mission_nr'])).intersection(set(metrics_hj_flocking_unfiltered['mission_nr']))
# Filter the rows in each dataframe that have matching mission_nr values
metrics_hj_naive = metrics_hj_naive_unfiltered[metrics_hj_naive_unfiltered['mission_nr'].isin(common_mission_nr)]
metrics_hj_reactive = metrics_hj_reactive_unfiltered[metrics_hj_reactive_unfiltered['mission_nr'].isin(common_mission_nr)]
metrics_hj_flocking = metrics_hj_flocking_unfiltered[metrics_hj_flocking_unfiltered['mission_nr'].isin(common_mission_nr)]

+ Sort by ascending missions nr order

In [5]:
metrics_hj_naive = metrics_hj_naive.sort_values(['mission_nr'])
metrics_hj_reactive= metrics_hj_reactive.sort_values(['mission_nr'])
metrics_hj_flocking = metrics_hj_flocking.sort_values(['mission_nr'])

+ Index by mission nr to get the same order in the panda dataframe between controller metrics

In [6]:
metrics_hj_naive.set_index('mission_nr', inplace=True)
metrics_hj_reactive.set_index('mission_nr', inplace=True)
metrics_hj_flocking.set_index('mission_nr', inplace=True)

+ Get Non Feasible Missions 

In [7]:
non_feas_idx = metrics_hj_naive.index[metrics_hj_naive["Reaching_target"] < 1]
ratio_non_feas_miss = len(non_feas_idx)/len(metrics_hj_naive)*100
print("Number of infeasible missions: ", len(non_feas_idx), f"corresponds to {ratio_non_feas_miss} %")

Number of infeasible missions:  43 corresponds to 7.846715328467154 %


+ Drop infeasible missions on HC

In [8]:
# metrics_hj_naive.drop(non_feas_idx, axis=0, inplace=True)
# metrics_hj_reactive.drop(non_feas_idx, axis=0, inplace=True)
# metrics_hj_flocking.drop(non_feas_idx, axis=0, inplace=True)

In [13]:
import plotly.express as px
import plotly.graph_objects as go

## Box Plots

In [16]:
from plotly.offline import  init_notebook_mode
init_notebook_mode(connected=True)

+ Beta Index

In [11]:
beta_index_avg_dic = {"hj_naive": metrics_hj_naive["Average Beta Index"],
                      "hj_reactive": metrics_hj_reactive["Average Beta Index"],
                      "flocking": metrics_hj_flocking["Average Beta Index"]}
beta_index_avg_df = pd.DataFrame(beta_index_avg_dic)
fig_beta = px.box(beta_index_avg_df)
mean_trace = go.Scatter(
    x=beta_index_avg_df.columns,
    y=beta_index_avg_df.mean(),
    mode='lines',
    line=dict(color='green', width=2),
    showlegend=True,
    name="mean"
)
fig_beta.add_trace(mean_trace)
# Add custom x and y labels
fig_beta.update_layout(
    xaxis_title="controllers",
    yaxis_title="Average Beta Index over Missions"
)
fig_beta.show()

+ Mean minimum distance to target among all platforms km

In [12]:
mean_min_dist_dict = {"hj_naive": Distance(deg=metrics_hj_naive["Mean minimum distance to target among all platforms in deg"]).km,
                      "hj_reactive": Distance(deg=metrics_hj_reactive["Mean minimum distance to target among all platforms in deg"]).km,
                      "flocking":Distance(deg=metrics_hj_flocking["Mean minimum distance to target among all platforms in deg"]).km}
idx_list = []
mean_min_dist_df = pd.DataFrame(mean_min_dist_dict)
fig_min_dist = px.box(mean_min_dist_df, log_y=True)
mean_trace = go.Scatter(
    x=mean_min_dist_df.columns,
    y=mean_min_dist_df.mean(),
    mode='lines',
    line=dict(color='green', width=2),
    showlegend=True,
    name="mean"
)
fig_min_dist.add_trace(mean_trace)
# Add custom x and y labels
fig_min_dist.update_layout(
    xaxis_title="controllers",
    yaxis_title=" [km]",
    title = {'text': "Mean minimum distance to target among all platforms"}
)
fig_min_dist.show()

+ Mean maximum correction from optimal control degrees

In [13]:
opt_ctrl_correction_dict = {"hj_naive": metrics_hj_naive["Mean maximum correction from optimal control degrees"],
                      "hj_reactive": metrics_hj_reactive["Mean maximum correction from optimal control degrees"],
                      "flocking": metrics_hj_flocking["Mean maximum correction from optimal control degrees"]}
opt_ctrl_correction_df = pd.DataFrame(opt_ctrl_correction_dict)
fig_opt_ctrl_correction = px.box(opt_ctrl_correction_df )
# Add custom x and y labels
fig_opt_ctrl_correction.update_layout(
    xaxis_title="controllers",
    yaxis_title="degrees",
    title = {'text': "Control direction correction from optimal input"}
)
fig_opt_ctrl_correction.show()

+ Isolated Platform Metric

In [14]:
ipm_dict= {"hj_naive": metrics_hj_naive["Isolated_platform_metric"],
                      "hj_reactive": metrics_hj_reactive["Isolated_platform_metric"],
                      "flocking": metrics_hj_flocking["Isolated_platform_metric"]}
ipm_df = pd.DataFrame(ipm_dict)
fig_ipm = px.box(ipm_df, log_y=False )
mean_trace = go.Scatter(
    x=ipm_df.columns,
    y=ipm_df.mean(),
    mode='lines',
    line=dict(color='green', width=2),
    showlegend=True,
    name="mean"
)
fig_ipm.add_trace(mean_trace)
# Add custom x and y labels
fig_ipm.update_layout(
    xaxis_title="controllers",
    yaxis_title="IPM",
    title = {'text': "Isolated_platform_metric"}
)
fig_ipm.show()

### Get Metrics summary as a dict:

In [8]:
def get_metrics_summary(metrics_file:pd.DataFrame, ctrl_name: str)->dict:
    return {
    "ctrl_name": ctrl_name,
    "avg_beta_index":metrics_file["Average Beta Index"].mean(),
    "avg_isolated_pltf_integral_hj_naive": metrics_file["Isolated_platform_metric"].mean(),
    "avg_correction_from_opt_ctrl_deg": metrics_file["Mean maximum correction from optimal control degrees"].mean(),
    "avg_mean_min_dist_to_target_km": Distance(deg=metrics_file["Mean minimum distance to target among all platforms in deg"].mean()).km,
    "avg_nb_platfroms_reaching_target": metrics_file["Reaching_target"].mean(),
    "nb_missions_with_isolated_pltf": np.sum(metrics_file["Isolated_platform_metric"] > 0),
    "nb_missions_with_collisions": np.sum(metrics_file["Number_of_collision"] > 0),
    "nb_missions_success": np.sum(metrics_file["Mission_success"]==1),
}

  + HJ Multi-Agent Naive

In [9]:
metrics_summary_hj_naive = get_metrics_summary(metrics_hj_naive, ctrl_name="hj_naive")
print(metrics_summary_hj_naive)
summary_hj_naive_df = pd.DataFrame(metrics_summary_hj_naive, index=[0])

{'ctrl_name': 'hj_naive', 'avg_beta_index': 1.0869452149791954, 'avg_isolated_pltf_integral_hj_naive': 0.3262182346580157, 'avg_correction_from_opt_ctrl_deg': 0.0, 'avg_mean_min_dist_to_target_km': 0.5073981506901873, 'avg_nb_platfroms_reaching_target': 0.9657846715328468, 'nb_missions_with_isolated_pltf': 262, 'nb_missions_with_collisions': 217, 'nb_missions_success': 125}


+ HJ Multi-Agent Decentralized Reactive Control

In [10]:
metrics_summary_hj_reactive = get_metrics_summary(metrics_hj_reactive, ctrl_name='hj_reactive')
print(metrics_summary_hj_reactive)
summary_hj_naive_reactive = pd.DataFrame(metrics_summary_hj_reactive, index=[0])

{'ctrl_name': 'hj_reactive', 'avg_beta_index': 1.2194893548093182, 'avg_isolated_pltf_integral_hj_naive': 0.11054845904298458, 'avg_correction_from_opt_ctrl_deg': 25.713171998879574, 'avg_mean_min_dist_to_target_km': 0.6602556119650705, 'avg_nb_platfroms_reaching_target': 0.9197080291970803, 'nb_missions_with_isolated_pltf': 157, 'nb_missions_with_collisions': 7, 'nb_missions_success': 337}


+ HJ Multi-Agent Flocking

In [11]:
metrics_summary_hj_flocking = get_metrics_summary(metrics_hj_flocking, ctrl_name='hj_flocking')
print(metrics_summary_hj_flocking)
summary_hj_naive_flocking= pd.DataFrame(metrics_summary_hj_flocking, index=[0])

{'ctrl_name': 'hj_flocking', 'avg_beta_index': 1.4108503244682469, 'avg_isolated_pltf_integral_hj_naive': 0.009014260610975934, 'avg_correction_from_opt_ctrl_deg': 24.359400797473548, 'avg_mean_min_dist_to_target_km': 0.5833058160461944, 'avg_nb_platfroms_reaching_target': 0.9411496350364964, 'nb_missions_with_isolated_pltf': 51, 'nb_missions_with_collisions': 7, 'nb_missions_success': 435}


## Confusion Matrices Plots

In [17]:
def plot_confusion_matrix(x, y, x_label, y_label, title_name:str):
    import seaborn as sns
    import matplotlib.pyplot as plt
    from matplotlib.colors import LogNorm

    confusion_matrix = pd.crosstab(x, y, rownames=[x_label], colnames=[y_label])
    fig = px.imshow(confusion_matrix, text_auto=True)
    fig.update_layout(
        title = {'text': title_name}
    )
    fig.show()
    # ax = sns.heatmap(confusion_matrix, annot=True, cmap=plt.cm.RdYlGn, square=True, fmt='g')
    # # ax = sns.heatmap(confusion_matrix, annot=True, cmap=plt.cm.RdYlGn, square=True, norm=LogNorm(), fmt='g') # if heat map is in norm
    # plt.tick_params(axis='both', which='major', labelbottom=False, bottom=False, top=True, labeltop=True)
    # ax.xaxis.set_label_position('top')
    # ax.plot
    # plt.title(title_name)
    # plt.show()
    return fig

###  HJ naive vs Decentralized Reactive Control

+  Missions with collisions

In [18]:
fig_conf_react_coll = plot_confusion_matrix(metrics_hj_naive["Number_of_collision"]>0, 
                        metrics_hj_reactive["Number_of_collision"]>0, 'Multi-Agent HJ Naive', 
                        'Multi-Agent HJ Reactive Control', title_name="Missions with collisions")

+ Missions with communication losses

In [19]:
fig_conf_react_comm_loss= plot_confusion_matrix(metrics_hj_naive["Isolated_platform_metric"]>0, 
                        metrics_hj_reactive["Isolated_platform_metric"]>0, 'Multi-Agent HJ Naive', 
                        'Multi-Agent HJ Reactive Control', title_name="Missions with communication losses")

+ Connectivity and Collision maintenance performance

In [20]:
connect_objective_reactive = (metrics_hj_reactive["Isolated_platform_metric"]==0) & (metrics_hj_reactive["Number_of_collision"] == 0)
connect_objective_hj_naive = (metrics_hj_naive["Isolated_platform_metric"]==0) & (metrics_hj_naive["Number_of_collision"] == 0)
fig_react_maint = plot_confusion_matrix(connect_objective_hj_naive , connect_objective_reactive , 
    'Multi-Agent HJ Naive', 'Multi-Agent HJ Reactive Control', title_name= 'Missions without collisions and with maintained connectivity')

+  Missions success counts

In [21]:
fig_react_success  = plot_confusion_matrix(metrics_hj_naive["Mission_success"]==1, 
                        metrics_hj_reactive["Mission_success"]==1, 'Multi-Agent HJ Naive', 
                        'Multi-Agent HJ Reactive', title_name="Mission Success")

###  HJ naive vs Flocking

+  Missions with collisions

In [22]:
fig_conf_flocking_coll= plot_confusion_matrix(metrics_hj_naive["Number_of_collision"]>0, 
                        metrics_hj_flocking["Number_of_collision"]>0, 'Multi-Agent HJ Naive', 
                        'Multi-Agent HJ Flocking Control', title_name="Missions with collisions")

+ Missions with communication losses

In [23]:
fig_conf_flocking_comm_loss = plot_confusion_matrix(metrics_hj_naive["Isolated_platform_metric"]>0, 
                        metrics_hj_flocking["Isolated_platform_metric"]>0, 'Multi-Agent HJ Naive', 
                        'Multi-Agent HJ Flocking Control', title_name="Missions with communication losses")

+ Connectivity and Collision maintenance performance

In [24]:
connect_objective_flocking = (metrics_hj_flocking["Isolated_platform_metric"]==0) & (metrics_hj_flocking["Number_of_collision"] == 0)
connect_objective_hj_naive = (metrics_hj_naive["Isolated_platform_metric"]==0) & (metrics_hj_naive["Number_of_collision"] == 0)
fig_flocking_maint = plot_confusion_matrix(connect_objective_hj_naive , connect_objective_flocking , 
    'Multi-Agent HJ Naive', 'Multi-Agent HJ Flocking Control', title_name= 'Missions without collisions and with maintained connectivity')

In [75]:
idx_failed_flocking = np.where((connect_objective_hj_naive==True) & (connect_objective_flocking==False))[0] # np where returns a tuple

In [70]:
metrics_hj_flocking.loc[idx_failed_flocking]

,Unnamed: 0,Isolated_platform_metric,Number_of_collision,Reaching_target,Mean minimum distance to target among all platforms in deg,Mean maximum correction from optimal control degrees,Average Beta Index,Initial maximum degree of the graph,Final maximum degree of the graph,Mission_success,mission_nr
15,15,0.044907,0,1.00,0.000000,13.982651,1.430652,2,3,0,86
48,48,0.014352,0,1.00,0.000000,75.308774,0.746186,2,2,0,59
85,85,0.013426,0,1.00,0.000000,38.388886,1.458391,2,3,0,51
190,190,1.000000,0,1.00,0.000000,49.386048,0.746533,2,2,0,426
258,258,0.000000,1,1.00,0.000000,11.177423,1.500000,3,3,0,362
333,333,0.014352,0,1.00,0.000000,17.037177,1.470874,2,3,0,288
351,351,0.010648,0,1.00,0.000000,81.244696,1.203537,2,3,0,284
393,393,0.000000,1,0.00,0.018569,15.144961,1.500000,3,3,0,226
395,395,0.807870,0,1.00,0.000000,40.458265,0.746879,2,3,0,213
487,487,0.115741,0,1.00,0.000000,76.455163,0.932039,2,1,0,12


In [56]:
metrics_hj_flocking

,Unnamed: 0,Isolated_platform_metric,Number_of_collision,Reaching_target,Mean minimum distance to target among all platforms in deg,Mean maximum correction from optimal control degrees,Average Beta Index,Initial maximum degree of the graph,Final maximum degree of the graph,Mission_success,mission_nr
547,547,0.000000,0,1.00,0.000000,10.403934,1.492718,3,3,1,0
426,426,0.000000,0,1.00,0.000000,8.973752,1.500000,3,3,1,1
332,332,0.021759,0,1.00,0.000000,16.015654,1.070388,2,3,0,2
218,218,0.000000,0,1.00,0.000000,23.109952,1.488211,3,3,1,3
105,105,0.000000,0,1.00,0.000000,6.288580,1.496186,3,3,1,4
...,...,...,...,...,...,...,...,...,...,...,...
53,53,0.000000,0,0.50,0.006600,22.882946,1.500000,3,3,0,546
56,56,0.000000,0,0.75,0.000931,6.194619,1.500000,3,3,0,547
55,55,0.000000,0,1.00,0.000000,63.464802,1.500000,3,3,1,548
54,54,0.000000,0,1.00,0.000000,22.030867,1.500000,3,3,1,549


In [72]:
metrics_hj_flocking.set_index('mission_nr', inplace=True)

In [73]:
metrics_hj_naive.set_index('mission_nr', inplace=True)

In [54]:
metrics_hj_naive.loc[idx_failed_flocking]

,Unnamed: 0,Isolated_platform_metric,Number_of_collision,Reaching_target,Mean minimum distance to target among all platforms in deg,Mean maximum correction from optimal control degrees,Average Beta Index,Initial maximum degree of the graph,Final maximum degree of the graph,Mission_success,mission_nr
15,15,0.0,0,1.00,0.00000,0.0,0.847087,2,3,1,84
48,48,0.0,0,1.00,0.00000,0.0,0.946949,2,3,1,58
85,85,0.0,0,0.75,0.00188,0.0,1.490291,3,3,0,517
190,190,0.0,0,1.00,0.00000,0.0,1.447295,3,3,1,425
258,258,0.0,0,1.00,0.00000,0.0,1.371706,3,3,1,357
333,333,0.0,0,1.00,0.00000,0.0,1.477809,3,3,1,292
351,351,0.0,0,1.00,0.00000,0.0,1.472261,3,3,1,282
393,393,0.0,0,1.00,0.00000,0.0,1.158807,3,3,1,243
395,395,0.0,0,1.00,0.00000,0.0,1.023232,3,3,1,261
487,487,0.0,0,1.00,0.00000,0.0,0.843620,3,3,1,142


In [43]:
metrics_hj_flocking[idx_failed_flocking[0]]

KeyError: "None of [Int64Index([15, 48, 85, 190, 258, 333, 351, 393, 395, 487, 496, 501], dtype='int64')] are in the [columns]"

+  Missions success counts

In [27]:
fig_flocking_success = plot_confusion_matrix(metrics_hj_naive["Mission_success"]==1, 
                        metrics_hj_flocking["Mission_success"]==1, 'Multi-Agent HJ Naive', 
                        'Multi-Agent HJ Flocking', title_name="Mission Success")

## Log Data in WandB

In [28]:
import wandb

In [29]:
# Log metrics in WandB
os.environ["WANDB_API_KEY"] = "1f19232e6ccc9981a8a972bee18ba31a94644835"

wandb.init(
    # Set the project where this run will be logged
    project="Master_Thesis_Run_Summaries",
    # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
    name=f"HC_HC_GOM_4_platforms",
    # Track hyperparameters and run metadata
    config={
        "Number of infeasible missions": len(non_feas_idx),
        "Percentage of infeasible missions": ratio_non_feas_miss,
        "Number of missions feasible": len(metrics_hj_naive),
    },
    entity="nhoischen",
)
wandb.log({"Metrics_Summary": wandb.Table(dataframe=pd.concat([summary_hj_naive_df, summary_hj_naive_reactive, summary_hj_naive_flocking]))},
            commit=False)
wandb.log({'Beta Index': fig_beta,
          'Time-Integral-over-Isolated-Platforms': fig_ipm,
          'Platforms-Mean-Min-Distance-to-Target': fig_min_dist,
           'Deviation-from-Cptimal-Ctrl-Direction': fig_opt_ctrl_correction,
            },
           commit=False)
wandb.log({'Missions with collisions hj-reactive': fig_conf_react_coll,
         'Missions with collisions hj-flocking': fig_conf_flocking_coll,
         'Missions with communication losses hj-reactive': fig_conf_react_comm_loss,
         'Missions with communication losses hj-flocking': fig_conf_flocking_comm_loss,
         'Missions with maintained connectivity and no collisions hj-reactive': fig_react_maint,
         'Missions with maintained connectivity and no collisions hj-flocking': fig_flocking_maint,
         'Succesfull Missions hj-reactive': fig_react_success,
         'Succesfull Missions hj-flocking': fig_flocking_success}, commit=True)
wandb.finish()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nhoischen. Use `wandb login --relogin` to force relogin
